# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 100
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('./dataset/cifar/train/*.png')[:100]
test_paths = glob('./dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Callbacks

In [14]:
start_time = datetime.now()
start_time

datetime.datetime(2020, 5, 13, 11, 6, 31, 564617)

In [15]:
datetime.now().strftime("%Y%m%d-%H%M%S")

'20200513-110631'

In [16]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))

In [17]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    write_graph=True,
    write_images=True,
    histogram_freq=1
)

In [ ]:
%tensorboard --logdir logs --host 10.178.144.232 --port 8010

#### tensorboard example
![tensorboard](image/tensorboard.jpg)

## Training

In [18]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

Epoch 1/100
3/3 [==============================] - 2s 543ms/step - loss: 2.3072 - accuracy: 0.0735 - val_loss: 2.3416 - val_accuracy: 0.0729
Epoch 2/100
3/3 [==============================] - 0s 122ms/step - loss: 2.2794 - accuracy: 0.1912 - val_loss: 2.3171 - val_accuracy: 0.1562
Epoch 3/100
3/3 [==============================] - 0s 122ms/step - loss: 2.4307 - accuracy: 0.0625 - val_loss: 2.3009 - val_accuracy: 0.1354
Epoch 4/100
3/3 [==============================] - 0s 118ms/step - loss: 2.2445 - accuracy: 0.0882 - val_loss: 2.2978 - val_accuracy: 0.1042
Epoch 5/100
3/3 [==============================] - 0s 122ms/step - loss: 2.2751 - accuracy: 0.1354 - val_loss: 2.3019 - val_accuracy: 0.0729
Epoch 6/100
3/3 [==============================] - 0s 118ms/step - loss: 2.1942 - accuracy: 0.1750 - val_loss: 2.3115 - val_accuracy: 0.1250
Epoch 7/100
3/3 [==============================] - 0s 119ms/step - loss: 2.1658 - accuracy: 0.2206 - val_loss: 2.3414 - val_accuracy: 0.0729
Epoch 8/100
3

Epoch 59/100
3/3 [==============================] - 0s 124ms/step - loss: 1.4331 - accuracy: 0.4706 - val_loss: 2.4079 - val_accuracy: 0.1771
Epoch 60/100
3/3 [==============================] - 0s 126ms/step - loss: 1.5946 - accuracy: 0.4271 - val_loss: 2.3990 - val_accuracy: 0.1771
Epoch 61/100
3/3 [==============================] - 0s 123ms/step - loss: 1.2726 - accuracy: 0.4559 - val_loss: 2.2904 - val_accuracy: 0.1979
Epoch 62/100
3/3 [==============================] - 0s 120ms/step - loss: 1.4513 - accuracy: 0.3676 - val_loss: 2.3197 - val_accuracy: 0.1979
Epoch 63/100
3/3 [==============================] - 0s 121ms/step - loss: 1.3666 - accuracy: 0.5104 - val_loss: 2.4416 - val_accuracy: 0.1875
Epoch 64/100
3/3 [==============================] - 0s 119ms/step - loss: 1.2929 - accuracy: 0.5294 - val_loss: 2.5333 - val_accuracy: 0.1771
Epoch 65/100
3/3 [==============================] - 0s 125ms/step - loss: 1.1284 - accuracy: 0.5147 - val_loss: 2.4423 - val_accuracy: 0.1875
Epoch 

### LambdaCallback

In [19]:
# https://www.tensorflow.org/tensorboard/r2/image_summaries

import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [20]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [21]:
# Define the per-epoch callback
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [22]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

Epoch 1/100
3/3 [==============================] - 1s 237ms/step - loss: 0.5365 - accuracy: 0.8382 - val_loss: 2.8721 - val_accuracy: 0.1875
Epoch 2/100
1/3 [=========>....................] - ETA: 0s - loss: 0.8038 - accuracy: 0.6562

/home/chanseok/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


3/3 [==============================] - 1s 213ms/step - loss: 0.4697 - accuracy: 0.7794 - val_loss: 3.0744 - val_accuracy: 0.2083
Epoch 3/100
3/3 [==============================] - 1s 230ms/step - loss: 0.7693 - accuracy: 0.7292 - val_loss: 2.9743 - val_accuracy: 0.1771
Epoch 4/100
3/3 [==============================] - 1s 213ms/step - loss: 0.5093 - accuracy: 0.8088 - val_loss: 3.1558 - val_accuracy: 0.2188
Epoch 5/100
3/3 [==============================] - 1s 199ms/step - loss: 0.5584 - accuracy: 0.7647 - val_loss: 3.2521 - val_accuracy: 0.2188
Epoch 6/100
3/3 [==============================] - 1s 203ms/step - loss: 0.3261 - accuracy: 0.8824 - val_loss: 3.2900 - val_accuracy: 0.1979
Epoch 7/100
3/3 [==============================] - 1s 206ms/step - loss: 0.7332 - accuracy: 0.7708 - val_loss: 3.2225 - val_accuracy: 0.2396
Epoch 8/100
3/3 [==============================] - 1s 208ms/step - loss: 0.4707 - accuracy: 0.7941 - val_loss: 3.1992 - val_accuracy: 0.2292


/home/chanseok/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


Epoch 9/100
1/3 [=========>....................] - ETA: 0s - loss: 0.1826 - accuracy: 1.0000

...

Epoch 60/100
3/3 [==============================] - 1s 207ms/step - loss: 0.2052 - accuracy: 0.8971 - val_loss: 5.3707 - val_accuracy: 0.2396
Epoch 61/100
3/3 [==============================] - 1s 200ms/step - loss: 0.2490 - accuracy: 0.8958 - val_loss: 4.9210 - val_accuracy: 0.2292
Epoch 62/100
3/3 [==============================] - 1s 262ms/step - loss: 0.3311 - accuracy: 0.8676 - val_loss: 4.0859 - val_accuracy: 0.2708
Epoch 63/100
3/3 [==============================] - 1s 200ms/step - loss: 0.2662 - accuracy: 0.9265 - val_loss: 3.9055 - val_accuracy: 0.2708
Epoch 64/100
3/3 [==============================] - 1s 206ms/step - loss: 0.2277 - accuracy: 0.8971 - val_loss: 4.2264 - val_accuracy: 0.2292
Epoch 65/100
3/3 [==============================] - 1s 206ms/step - loss: 0.5363 - accuracy: 0.9412 - val_loss: 4.1596 - val_accuracy: 0.2500
Epoch 66/100
3/3 [==============================] - 1s 205ms/step - loss: 0.2617 - accuracy: 0.8958 - val_loss: 4.1585 - val_accuracy: 0.2917
Epoch 

/home/chanseok/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


3/3 [==============================] - 1s 212ms/step - loss: 0.0527 - accuracy: 0.9853 - val_loss: 5.2492 - val_accuracy: 0.2500
Epoch 100/100
1/3 [=========>....................] - ETA: 0s - loss: 0.0038 - accuracy: 1.0000

/home/chanseok/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


3/3 [==============================] - 1s 209ms/step - loss: 0.0483 - accuracy: 0.9706 - val_loss: 5.4912 - val_accuracy: 0.2604


## Expert

In [23]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [24]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [25]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [26]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [27]:
file_writer = tf.summary.create_file_writer(logdir)

In [29]:
from tqdm.notebook import tqdm

In [ ]:
for epoch in tqdm(range(num_epochs)):
    for step, (images, labels) in enumerate(train_dataset):
        train_step(images, labels)
        
    with file_writer.as_default():
        tf.summary.images('input_image', images, step=step + (step * epoch))
        tf.summary.scalar('train_loss', train_loss.result(), step=step + (step * epoch))
        tf.summary.scalar('train_accuracy', train_accuracy.result(), step=step + (step * epoch))
        
    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)
        
    template = 'Epoch {}. Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100,
                                     test_loss.result(), test_accuracy.result() * 100))